# Partie 2

Chargement des données depuis le fichier CSV

In [ ]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/ankitshah009/Task-4-Large-scale-weakly-supervised-sound-event-detection-for-smart-cars/master/groundtruth_weak_label_training_set.csv', 
                   sep='\t', 
                   error_bad_lines=False,
                   names=["file", "start", "end", "label"])
data


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,file,start,end,label
0,2ceUOv8A3FE_20.000_30.000.wav,20.0,30.0,Train horn
1,2hcY8kXQj60_30.000_40.000.wav,30.0,40.0,Train horn
2,2pt7WZBYeeI_30.000_40.000.wav,30.0,40.0,Train horn
3,2wG6EBWr5NQ_30.000_40.000.wav,30.0,40.0,Train horn
4,2wadsgHvSAM_240.000_250.000.wav,240.0,250.0,Train horn
...,...,...,...,...
56126,zjHUFwrJbpc_0.000_10.000.wav,0.0,10.0,Train
56127,zjQsLnmGJn4_60.000_70.000.wav,60.0,70.0,Train
56128,zoeGsPS6ovY_110.000_120.000.wav,110.0,120.0,Train
56129,ztdPIQeG8nw_30.000_40.000.wav,30.0,40.0,Train


In [3]:
# -*- coding: utf-8 -*-
"""
Script to learn a model with Scikit-learn.

Created on Mon Oct 24 20:51:47 2022

@author: ValBaron10
"""

import os
import numpy as np
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import pickle
from joblib import dump
from features_functions import compute_features

In [32]:
import wave

def load_wave(filepath):

  with wave.open(filepath, "rb") as file:
    audio = file.readframes(file.getnframes())

    # Convert buffer to float32 using NumPy                                                                                 
    audio_as_np_int16 = np.frombuffer(audio, dtype=np.int16)
    audio_as_np_float32 = audio_as_np_int16.astype(np.float32)

    # Normalise float32 array so that values are between -1.0 and +1.0                                                      
    max_int16 = 2**15
    audio_normalised = audio_as_np_float32 / max_int16
    return audio_normalised

In [35]:
labels = ("cars", "trucks")
learningFeatures = False
learningLabels = False


for label in labels:

  folder = f"data_part_2/{label}"
  for file in os.scandir(folder): 
    
    if not file.name.endswith(".wav"): continue

    input_sig = load_wave(folder + "/" + file.name)

    # Compute the signal in three domains
    sig_sq = input_sig**2
    sig_t = input_sig / np.sqrt(sig_sq.sum())
    sig_f = np.absolute(np.fft.fft(sig_t))
    sig_c = np.absolute(np.fft.fft(sig_f))

    # Compute the features and store them
    features_list = []
    N_feat, features_list = compute_features(sig_t, sig_f[:sig_t.shape[0]//2], sig_c[:sig_t.shape[0]//2])
    features_vector = np.array(features_list)[np.newaxis,:]

    # Store the obtained features in a np.arrays
    if type(learningFeatures) == bool: learningFeatures = features_vector.copy()
    else: learningFeatures = np.append(learningFeatures, features_vector, axis=0)
  
    if type(learningLabels) == bool: learningLabels = np.array([label])
    else: learningLabels = np.append(learningLabels, [label])

    

# Encode the class names
labelEncoder = preprocessing.LabelEncoder().fit(learningLabels)
learningLabelsStd = labelEncoder.transform(learningLabels)



In [36]:
# Learn the model
model = svm.SVC(C=10, kernel='linear', class_weight=None, probability=False)
scaler = preprocessing.StandardScaler(with_mean=True).fit(learningFeatures)
learningFeatures_scaled = scaler.transform(learningFeatures)


x_train, x_test, y_train, y_test = train_test_split(
    learningFeatures_scaled, learningLabelsStd, test_size=0.3, shuffle=False
)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(
    f"Classification report for classifier {model}:\n"
    f"{classification_report(y_test, y_pred)}\n"
)


# Export the scaler and model on disk
dump(scaler, "SCALER")
dump(model, "SVM_MODEL")

Classification report for classifier SVC(C=10, kernel='linear'):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.67      0.80         9

    accuracy                           0.67         9
   macro avg       0.50      0.33      0.40         9
weighted avg       1.00      0.67      0.80         9




/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['SVM_MODEL']